In [181]:
using Pkg
Pkg.activate("/home/fedflorio/master_thesis")

  Activating project at `~/master_thesis`


In [182]:
using Revise
using MatrixProductBP, MatrixProductBP.Models
using Graphs, Printf, IndexedGraphs, Statistics, Random
import ProgressMeter; ProgressMeter.ijulia_behavior(:clear)
using JLD2
using TensorTrains: summary_compact
using SparseArrays;

In [183]:
T = 200
N = 30
seed = 2

c = 2
gg = erdos_renyi(N, c/N; seed)
g = IndexedGraph(gg)

λ_unif = 0.5
ρ_unif = 0.2
λ = zeros(N,N)
for i in CartesianIndices(λ)
    if !iszero(g.A[i])
        λ[i] = λ_unif
    end
end
λ = sparse(λ)
ρ = fill(ρ_unif,N)
γ = 0.5

# T = 3
# N = 2
# seed = 6

# A = [0 1; 1 0]
# g = IndexedGraph(A)

# λ_unif = 0.7
# ρ_unif = 0.1
# λ = sparse(λ_unif .* A)
# # λ = sparse([0 0; λ_unif 0])
# ρ = fill(ρ_unif, N)
# γ = 0.5

sis = SIS_heterogeneous(λ, ρ, T; γ);
bp = mpbp(sis);

In [184]:
g.A

30×30 SparseMatrixCSC{Int64, Int64} with 52 stored entries:
⎡⠀⠀⠀⠀⠈⠀⠈⠀⠈⠀⠈⠀⠠⠀⠀⎤
⎢⠀⠀⢀⠐⠐⠀⢀⠬⠀⠀⢀⠀⠠⠀⠀⎥
⎢⠂⠀⠐⠀⠠⠂⠰⠀⠈⠀⠁⡀⠀⠀⠀⎥
⎢⠂⠀⡀⡔⠐⠂⠀⠀⠀⠄⠁⠀⠀⢐⠀⎥
⎢⠂⠀⠀⠀⠂⠀⠀⠄⠀⠀⠀⠈⠀⠀⠀⎥
⎢⠂⠀⠀⠐⠁⠠⠁⠀⡀⠀⠀⠀⠀⠄⡀⎥
⎢⠀⠂⠀⠂⠀⠀⢀⢀⠀⠀⠀⠄⠀⠀⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠀⠀⠀⎦

In [185]:
obs_times = collect(0:2:T)
nobs = floor(Int, N * length(obs_times) * 1.0)
obs_fraction = nobs / N
rng = MersenneTwister(seed)
X, observed = draw_node_observations!(bp, nobs, times = obs_times .+ 1, softinf=Inf; rng);

In [186]:
X

30×201 Matrix{Int64}:
 1  2  2  1  2  2  2  1  2  1  2  2  2  …  2  2  2  2  2  2  2  2  2  1  2  2
 2  2  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  2  1  2  2  2  2  2  2  2  2  2  2     2  2  2  2  1  2  2  2  2  2  2  2
 2  2  2  2  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 2  2  2  2  2  1  2  2  2  1  2  1  1     2  2  2  2  2  1  2  2  2  1  1  1
 2  1  2  2  1  2  2  2  2  2  1  1  1  …  1  1  1  1  1  1  2  1  2  2  2  2
 2  1  2  2  2  2  1  2  2  1  2  2  1     2  2  2  2  2  2  2  1  2  1  2  2
 1  2  2  1  2  2  2  2  1  2  2  1  1     1  2  2  2  2  2  2  2  2  2  2  2
 2  2  2  2  1  2  2  2  2  2  2  2  2     2  1  2  1  1  2  2  2  2  2  2  2
 2  1  2  2  2  2  2  2  2  2  1  2  2     1  1  1  2  1  2  2  2  2  1  2  2
 1  1  1  2  2  2  2  2  2  1  1  2  2  …  1  1  1  2  2  2  2  2  1  1  1  1
 2  2  2  2  2  1  1  1  1  2  2  2  2     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  2  2  2  2  2  2  1  2  1    

In [187]:
reset_messages!(bp)
reset_beliefs!(bp)
svd_trunc = TruncBond(5)

iters, = iterate!(bp, maxiter=5; svd_trunc, tol=1e-12);

Running MPBP: iter 5    Time: 0:00:22
  Δ:                0.0013134248236306156


In [188]:
obs_node = 1
# for t in eachindex(bp.w[obs_node])
#     bp.w[obs_node][t].λ .= nonzeros(λ)[nzrange(λ,obs_node)]
# end

der_mpbp = der_λ(bp, obs_node, eltype(bp.w[obs_node]); svd_trunc)

4-element Vector{Float64}:
 -0.05497196879327216
  0.14864402564192594
 -0.17096338409058978
 -0.09532908412664233

In [189]:
ϵ = 1e-8

logzᵢ0 = onebpiter!(bp, obs_node, eltype(bp.w[obs_node]); svd_trunc)

for t in eachindex(bp.w[obs_node])
    bp.w[obs_node][t].λ[1] += ϵ
end

logzᵢϵ = onebpiter!(bp, obs_node, eltype(bp.w[obs_node]); svd_trunc)

for i in vertices(g)
    for t in eachindex(bp.w[i])
        bp.w[i][t].λ .= nonzeros(λ)[nzrange(λ,i)]
    end
end

der_num = (logzᵢϵ - logzᵢ0)/ϵ 

-0.055001692089717835